In [1]:
4+ 7

11

In [ ]:
%%writefile matrixMultiplication.cu
#include <iostream>
#include <math.h>
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <assert.h>
#include <curand.h>
#include <curand_kernel.h>
#define N 4000

inline cudaError_t checkCuda(cudaError_t result)
{
  if (result != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime Error: %s\n", cudaGetErrorString(result));
    assert(result == cudaSuccess);
  }
  return result;
}

// CUDA kernel to multiply matrices
__global__ void mult(float **x, float **y, float **z){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    // stride not memory coalescent; might need to fix
    for (int i = index; i < N; i += stride) {
        for (int j = 0; j < N; j++) {
            float sum = 0;
            for (int k = 0; k < N; k++) {
                sum += x[i][k] * y[k][j];
            }
            z[i][j] = sum;
        }
    }
}

// CUDA kernel to init the matrices
// NOT USED SINCE RAND() DOESN'T WORK IN KERNEL FUNCTIONS
//__global__ void mult(float **x, float **y){
//    int index = blockIdx.x * blockDim.x + threadIdx.x;
//    int stride = blockDim.x * gridDim.x;
//    // stride not memory coalescent; might need to fix
//    for (int i = index; i < N; i += stride) {
//        for (int j = 0; j < N; j++) {
//            x[i][j] = (float) rand() / RAND_MAX;
//            y[i][j] = (float) rand() / RAND_MAX;
//        }
//    }
//}

int main(void){
    float **x, **y, **z;
    // Allocate Unified Memory -- accessible from CPU or GPU
    checkCuda(cudaMallocManaged(&x, N*sizeof(float *)));
    checkCuda(cudaMallocManaged(&y, N*sizeof(float *)));
    checkCuda(cudaMallocManaged(&z, N*sizeof(float *)));

    for (int i = 0; i < N; i++) {
        checkCuda(cudaMallocManaged(&y[i], N*sizeof(float)));
        checkCuda(cudaMallocManaged(&x[i], N*sizeof(float)));
        checkCuda(cudaMallocManaged(&z[i], N*sizeof(float)));
        //printf("%d\n", i);
    }

    // initialize x and y arrays on the host
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            y[i][j] = (float) rand() / RAND_MAX;
            x[i][j] = (float) rand() / RAND_MAX;
        }
    }


    //for timing
    float milliseconds=0.0;
    cudaEvent_t start,stop;
    checkCuda(cudaEventCreate(&start));
    checkCuda(cudaEventCreate(&stop));


    // Launch kernel on 1M elements on the GPU
    int blockSize = 100;
    int numBlocks = 256;
    cudaEventRecord(start);
    mult<<<numBlocks, blockSize>>>(x, y, z);

    // Wait for GPU to finish before accessing on host
    checkCuda(cudaEventRecord(stop));
    checkCuda(cudaDeviceSynchronize());

    checkCuda(cudaEventElapsedTime(&milliseconds, start, stop));
    std::cout << "Elapsed time for Cuda matrix multiplication: " << milliseconds/1000 << std::endl;

    // Free memory
    for (int i = 0; i < N; i++) {
        checkCuda(cudaFree(x[i]));
        checkCuda(cudaFree(y[i]));
        checkCuda(cudaFree(z[i]));
    }
    checkCuda(cudaFree(x));
    checkCuda(cudaFree(y));
    checkCuda(cudaFree(z));


    return 0;
}

Overwriting matrixMultiplication.cu


In [ ]:
!nvcc  -o matrixMultiplication matrixMultiplication.cu

In [ ]:
!./matrixMultiplication

Elapsed time for Cuda matrix multiplication: 4.46919


In [ ]:
%%writefile c_implementation.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <sys/time.h>
#include <pthread.h>
#include <stdint.h>
#include <time.h>
#include <omp.h>

// #define _POSIX_C_SOURCE >= 199309L
#define CORE 4
#define HEIGHT1 4000
#define WIDTH1 4000
#define HEIGHT2 4000
#define WIDTH2 4000

float CMat[HEIGHT1][WIDTH1], DMat[HEIGHT2][WIDTH2], multPthread[HEIGHT1][WIDTH2], multSeq[HEIGHT1][WIDTH2], multOpenMP[HEIGHT1][WIDTH2];
pthread_t thread[CORE];
struct timespec begin, end;



void *multMatrices_openmp()
{
    int row, col, k;

    #pragma omp parallel num_threads(4) shared(multOpenMP, CMat, DMat) private(row, col, k)
    {

    #pragma omp for
    for (row = 0; row < HEIGHT1; row++)
    {
        for (col = 0; col < WIDTH2; col++)
        {
            for (k = 0; k < WIDTH1; k++)
            {
                multOpenMP[row][col] += CMat[row][k] * DMat[k][col];
            }
        }
    }
    }

    return NULL;
}

void *multMatrices_pthread(void *arg)
{
    int row, col, k;
    float val = 0;

    int core = (intptr_t)arg;
    // Each thread computes 1/core of matrix multiplication
    for (row = core * HEIGHT1 / CORE; row < (core + 1) * HEIGHT1 / CORE; row++)
    {
        for (col = 0; col < WIDTH2; col++)
        {
            val = 0;
            for (k = 0; k < WIDTH1; k++)
            {

                val += CMat[row][k] * DMat[k][col];
            }

            multPthread[row][col] = val;
        }
    }

    return NULL;
}

int main(int argc, char *argv[])
{
    int i, j;
    int err_val;
    int num_errors = 0;
    double timeSeq, timePthread, timeOpenMP;

    int row, col, k;
    float val = 0;

    for (i = 0; i < HEIGHT1; i++)
    {
        for (j = 0; j < WIDTH1; j++)
        {
            CMat[i][j] = (float)rand() / RAND_MAX;
        }
    }

    for (i = 0; i < HEIGHT2; i++)
    {
        for (j = 0; j < WIDTH2; j++)
        {
            DMat[i][j] = (float)rand() / RAND_MAX;
        }
    }


    err_val = clock_gettime(CLOCK_MONOTONIC, &begin);
    if (err_val != 0) {
        fprintf(stderr, "timing failure in %s at line %d\n", __FILE__, __LINE__);
        perror(NULL);
        exit(EXIT_FAILURE);
    }
    // calculate sequential product
    for (row = 0; row < HEIGHT1; row++)
    {
        for (col = 0; col < WIDTH2; col++)
        {
            val = 0;
            for (k = 0; k < WIDTH1; k++)
            {
                val += CMat[row][k] * DMat[k][col];
            }

            multSeq[row][col] = val;
        }
    }
    err_val = clock_gettime(CLOCK_MONOTONIC, &end);
    if (err_val != 0) {
        fprintf(stderr, "timing failure in %s at line %d\n", __FILE__, __LINE__);
        perror(NULL);
        exit(EXIT_FAILURE);
    }

    timeSeq = end.tv_sec - begin.tv_sec;
    timeSeq += (end.tv_nsec - begin.tv_nsec) / 1000000000.0;

    printf("Time sequential: %f\n", timeSeq);



    // pthread implementation
    err_val = clock_gettime(CLOCK_MONOTONIC, &begin);
    if (err_val != 0) {
        fprintf(stderr, "timing failure in %s at line %d\n", __FILE__, __LINE__);
        perror(NULL);
        exit(EXIT_FAILURE);
    }
    for (i = 0; i < CORE; i++)
    {
        err_val = pthread_create(&thread[i], NULL, &multMatrices_pthread, (void *)(intptr_t)i);
        if (err_val != 0)
        {
            fprintf(stderr, "pthread create failure in %s at line %d\n", __FILE__, __LINE__);
            perror(NULL);
            exit(EXIT_FAILURE);
        }
    }

    for (i = 0; i < CORE; i++)
    {
        err_val = pthread_join(thread[i], NULL);
        if (err_val != 0)
        {
            fprintf(stderr, "pthread join failure in %s at line %d\n", __FILE__, __LINE__);
            perror(NULL);
            exit(EXIT_FAILURE);
        }
    }
    err_val = clock_gettime(CLOCK_MONOTONIC, &end);
    if (err_val != 0) {
        fprintf(stderr, "timing failure in %s at line %d\n", __FILE__, __LINE__);
        perror(NULL);
        exit(EXIT_FAILURE);
    }

    timePthread = end.tv_sec - begin.tv_sec;
    timePthread += (end.tv_nsec - begin.tv_nsec) / 1000000000.0;

    for (i = 0; i < HEIGHT1; i++) {
        for (j = 0; j < WIDTH2; j++) {
            if (multSeq[i][j] != multPthread[i][j]) {
                num_errors++;
            }
        }
    }
    
    if (num_errors != 0)
    {
        printf("Multiplication pthread: Failure\n");
        printf("Num errors: %d\n", num_errors);
        return 1;
    }
    else
    {
        printf("Multiplication pthread: Success!!!\n");
        printf("Time pthread: %f\n", timePthread);
    }



    // openMP implementation 1
    num_errors = 0;
    err_val = clock_gettime(CLOCK_MONOTONIC, &begin);
    if (err_val != 0) {
        fprintf(stderr, "timing failure in %s at line %d\n", __FILE__, __LINE__);
        perror(NULL);
        exit(EXIT_FAILURE);
    }

    multMatrices_openmp();

    err_val = clock_gettime(CLOCK_MONOTONIC, &end);
    if (err_val != 0) {
        fprintf(stderr, "timing failure in %s at line %d\n", __FILE__, __LINE__);
        perror(NULL);
        exit(EXIT_FAILURE);
    }
    
    timeOpenMP = end.tv_sec - begin.tv_sec;
    timeOpenMP += (end.tv_nsec - begin.tv_nsec) / 1000000000.0;

    for (i = 0; i < HEIGHT1; i++) {
        for (j = 0; j < WIDTH2; j++) {
            if (multSeq[i][j] != multOpenMP[i][j]) {
                num_errors++;
            }
        }
    }

    if (num_errors != 0)
    {
        printf("Multiplication OpenMP 1: Failure\n");
        printf("Num errors: %d\n", num_errors);
        return 1;
    }
    else
    {
        printf("Multiplication OpenMP 1: Success!!!\n");
        printf("Time OpenMP: %f\n", timeOpenMP);
    }
    
    return 0;
}


Writing c_implementation.c


In [ ]:
!gcc c_implementation.c -O3 -lpthread -fopenmp -o c_implementation

In [ ]:
!./c_implementation

Time sequential: 132.980816
Multiplication pthread: Success!!!
Time pthread: 123.727424
Multiplication OpenMP 1: Success!!!
Time OpenMP: 123.704862


In [ ]:
%%writefile matrixMultiplicationtest.cu
#include <iostream>
#include <math.h>
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <assert.h>
#include <curand.h>
#include <curand_kernel.h>
# define N 100

inline cudaError_t checkCuda(cudaError_t result)
{
  if (result != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime Error: %s\n", cudaGetErrorString(result));
    assert(result == cudaSuccess);
  }
  return result;
}

// CUDA kernel to multiply matrices
__global__ void mult(float **x, float **y, float **z){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    // stride not memory coalescent; might need to fix
    for (int i = index; i < N; i += stride) {
        for (int j = 0; j < N; j++) {
            float sum = 0;
            for (int k = 0; k < N; k++) {
                sum += x[i][k] * y[k][j];
            }
            z[i][j] = sum;
        }
    }
}

// CUDA kernel to init the matrices
//__global__ void mult(float **x, float **y){
//    int index = blockIdx.x * blockDim.x + threadIdx.x;
//    int stride = blockDim.x * gridDim.x;
//    // stride not memory coalescent; might need to fix
//    for (int i = index; i < N; i += stride) {
//        for (int j = 0; j < N; j++) {
//            x[i][j] = (float) rand() / RAND_MAX;
//            y[i][j] = (float) rand() / RAND_MAX;
//        }
//    }
//}

// check that our cuda multiplication algorithm is correct
int main(void){
    float **x, **y, **z, **multSeq;
    int row, col, k, num_errors;
    float val;

    // Allocate Unified Memory -- accessible from CPU or GPU
    checkCuda(cudaMallocManaged(&x, N*sizeof(float *)));
    checkCuda(cudaMallocManaged(&y, N*sizeof(float *)));
    checkCuda(cudaMallocManaged(&z, N*sizeof(float *)));
    checkCuda(cudaMallocManaged(&multSeq, N*sizeof(float *)));

    for (int i = 0; i < N; i++) {
        checkCuda(cudaMallocManaged(&y[i], N*sizeof(float)));
        checkCuda(cudaMallocManaged(&x[i], N*sizeof(float)));
        checkCuda(cudaMallocManaged(&z[i], N*sizeof(float)));
        checkCuda(cudaMallocManaged(&multSeq[i], N*sizeof(float)));
    }

    // initialize matrices on GPU because faster (less data movement from CPU to GPU if data exists on GPU)
    // initialize x and y arrays on the host
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            y[i][j] = (float) rand() / RAND_MAX;
            x[i][j] = (float) rand() / RAND_MAX;
        }
    }

    for (row = 0; row < N; row++)
    {
        for (col = 0; col < N; col++)
        {
            val = 0;
            for (k = 0; k < N; k++)
            {
                val += x[row][k] * y[k][col];
            }

            multSeq[row][col] = val;
        }
    }

    //for timing
    float milliseconds=0.0;
    cudaEvent_t start,stop;
    checkCuda(cudaEventCreate(&start));
    checkCuda(cudaEventCreate(&stop));

    // Launch kernel on 1M elements on the GPU
    int blockSize = 100;
    int numBlocks = 100;
    checkCuda(cudaEventRecord(start));
    mult<<<numBlocks, blockSize>>>(x, y, z);

    // Wait for GPU to finish before accessing on host
    checkCuda(cudaEventRecord(stop));
    checkCuda(cudaDeviceSynchronize());

    checkCuda(cudaEventElapsedTime(&milliseconds, start, stop));
    std::cout << "Cuda elapsed time: " << milliseconds/1000 << std::endl;

    num_errors = 0;
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            // handling float precision
            if (abs(multSeq[i][j] - z[i][j]) > 0.001) {
                num_errors++;
                printf("value 1 = %f value 2 = %f\n", multSeq[i][j], z[i][j]);
            }
        }
    }

    if (num_errors != 0)
    {
        printf("Multiplication Cuda: Failure\n");
        printf("Num errors: %d\n", num_errors);
        //return 1;
    }
    else
    {
        printf("Multiplication Cuda: Success!!!\n");
        //printf("Time OpenMP: %f\n", timeOpenMP);
    }
    
    // Free memory
    for (int i = 0; i < N; i++) {
        checkCuda(cudaFree(x[i]));
        checkCuda(cudaFree(y[i]));
        checkCuda(cudaFree(z[i]));
        checkCuda(cudaFree(multSeq[i]));
    }
    checkCuda(cudaFree(x));
    checkCuda(cudaFree(y));
    checkCuda(cudaFree(z));
    checkCuda(cudaFree(multSeq));

    return 0;
}

Overwriting matrixMultiplicationtest.cu


In [ ]:
!nvcc  -o matrixMultiplicationtest matrixMultiplicationtest.cu

In [ ]:
!./matrixMultiplicationtest

Cuda elapsed time: 0.00263782
Multiplication Cuda: Success!!!


# Timings

### Pthread, sequential and openMP timings taken on 127x20, CUDA timings taken on colab 

Timings for CUDA-

* N = 4000 and <<<256, 100>>> - 4.3152
* N = 4000 and <<<1024, 100>>> - 4.53956
* N = 10000 and <<<256, 100>>> - 63.0042
* N = 10000 and <<<1024, 100>>> - 64.1735


Timings for 4 threads (N = 4000)-


*   OpenMP -  74.498309
*   Pthread - 74.690056
*   Sequential - 305.769212

Timings for 2 threads (N = 4000)-


*   OpenMP -  158.567216
*   Pthread - 163.935921
*   Sequential - 304.99212